In [ ]:
# install AML SDK
pip install --upgrade azureml-sdk

In [ ]:
# install CMAKS SDK
pip install --disable-pip-version-check --extra-index-url https://azuremlsdktestpypi.azureedge.net/CmAks-Compute-Test/D58E86006C65 azureml-pipeline-steps azureml-contrib-pipeline-steps azureml_contrib_itp --upgrade

In [ ]:
# install SDK 2.0
!pip install azureml-pipeline-wrapper[steps,notebooks]==0.1.0.15254362 --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/15254362 --user --upgrade

In [ ]:
# define resource used
workspace_name = <your workspace name>
sub_id = <sub id>
resource_group = <resource group>
cmaks_compute = <cmaks compute>

In [ ]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

# change to your ws space info
ws = Workspace.get(name=workspace_name, subscription_id=sub_id, resource_group=resource_group)

train_module_func = Module.from_yaml(ws, './train-score-eval/train.yaml')
score_module_func = Module.from_yaml(ws, './train-score-eval/score.yaml')
eval_module_func = Module.from_yaml(ws, './train-score-eval/eval.yaml')
compare_module_func = Module.from_yaml(ws, './train-score-eval/compare2.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [ ]:
# define a sub pipeline
@dsl.pipeline(name = 'Train-Score-Eval', 
              description = 'train model and evaluate model perf')
def training_pipeline(input_data, learning_rate):
    train = train_module_func(
        training_data=input_data, 
        max_epochs=5, 
        learning_rate=learning_rate)
   
    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)

    return {'eval_output': eval.outputs.eval_output, 'model_output': train.outputs.model_output}

In [ ]:
# import cmakscompute, if don't, can't find cmaks compute
from azureml.contrib.core.compute.cmakscompute import CmAksCompute

In [ ]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'Dummy automl pipeline - aks', 
              description = 'select best model trained with different learning rate',
              default_compute_target = cmaks_compute)
def dummy_automl_pipeline():
    train_and_evalute_model1 = training_pipeline(train_data, 0.01)
    train_and_evalute_model2 = training_pipeline(train_data, 0.02)
    
    compare = compare_module_func(
        model1=train_and_evalute_model1.outputs.model_output, 
        eval_result1=train_and_evalute_model1.outputs.eval_output,
        model2=train_and_evalute_model2.outputs.model_output,
        eval_result2=train_and_evalute_model2.outputs.eval_output
    )

    return {**compare.outputs}

# create a pipeline
pipeline = dummy_automl_pipeline()

In [ ]:
#pipeline.validate()

In [ ]:
pipeline.submit(experiment_name='samples')